In [ ]:
#!pip install tsfresh

In [ ]:
import pandas as pd
import datetime as dt
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
cluster1_path = 'cluster1.csv' 
cluester1 = pd.read_csv(cluster1_path)
cluester1.drop(columns = ['Unnamed: 0'],inplace = True)

In [ ]:
cluester1

In [ ]:
dict_label = {
            'act_1' : 0,
            'act_2': 1
            }

In [ ]:
cluester1['label']= cluester1['label'].map(dict_label) 

In [ ]:
cluester1["date"] = pd.to_datetime(cluester1["date"],format = '%Y-%m-%d')

In [ ]:
cluester1.sort_values(['pair','date'], inplace = True)

In [ ]:
cluester1['pair_index_ant'] = cluester1.groupby('pair')['pair_index'].shift(1)

In [ ]:
cluester1['retorno'] = cluester1['pair_index']/cluester1['pair_index_ant']-1

In [ ]:
cluester1.head()

In [ ]:
cluester1_x = cluester1[['pair','date','retorno']].copy()

In [ ]:
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from sklearn.ensemble import AdaBoostRegressor
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.utilities.dataframe_functions import roll_time_series
from tsfresh import select_features

In [ ]:
x_1 = cluester1_x[cluester1_x['pair']=='EWH US Equity_EWY US Equity'].dropna().drop(columns = ['pair']).copy()

In [ ]:
tsfresh_feautures = pd.DataFrame()
for k, v in cluester1_x.groupby('pair'):
    aux = cluester1_x[cluester1_x['pair']==k].dropna().drop(columns = ['pair']).copy()
    aux.set_index('date', inplace = True)
    aux_2 = aux['retorno'].copy()
    df_shift, y = make_forecasting_frame(aux_2, kind="retorno", max_timeshift=50, rolling_direction=1)
    aux_3 = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute, 
                     show_warnings=False)
    aux_3['pair'] = k
    aux_3.reset_index(inplace = True)
    aux_3.rename(columns = {'id':'date'},inplace = True)
#    aux_3['date']=aux_3['date'].shift(1) # para mover las caracteristicas un dia atras.
    tsfresh_feautures = pd.concat([tsfresh_feautures,aux_3])

In [ ]:
#df_shift
#tsfresh_feautures.rename(columns = {'id':'date'},inplace = True)
tsfresh_feautures.shape

In [ ]:
cluester1_all = cluester1.merge(tsfresh_feautures, how = 'left', on = ['date','pair'])

In [ ]:
cluester1_all_no = cluester1_all.drop(columns = ['pair_index']).dropna()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import GradientBoostingClassifier
#%%

models = {
            'linear' : {
                    'mod' : LogisticRegression(),
                    'par' : {'solver' : ('newton-cg', 'lbfgs'),
                                 'class_weight' : (None, 'balanced')}
                    },                     
            'gradient' : {
                    'mod' : GradientBoostingClassifier(warm_start = True),
                    'par' : {'loss' : ('deviance', 'exponential'),
                             'max_depth' : [3, 4, 5, 6, 7]}
                        },
            'SVM' : {
                    'mod' : SVC(gamma='auto',probability=True),
                    'par' : {'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 'C':[1, 10]}
                    },
            'Random_Forest' : {
                    'mod' : RandomForestClassifier(random_state=42),
                    'par' : {    'n_estimators': [100, 300],'max_features': ['auto', 'sqrt', 'log2'],'max_depth' : [4,5,6],
                    'criterion' :['gini', 'entropy']}
                    },
            }

In [ ]:
def grid(now_date, n_proc, os_X_tt, os_Y_tt, 
         models, score = 'roc_auc', cv = 5):
    # Gridsearch
    
    for name in models:
        print('*'*80)
        print("Model: " + name)
        t_beg = time.time()

        pipeline = Pipeline([('scaler', StandardScaler()), (name,  models[name]['mod'])])          
        parameters = {}          
        for par in models[name]['par']:
            aux = name + '__' +  par
            parameters[aux] = models[name]['par'][par]    
        aux = GridSearchCV(pipeline, parameters, n_jobs = n_proc,\
                          scoring = score, verbose=2, cv = cv)
        aux.fit(os_X_tt, os_Y_tt)
        models[name]['bestModel'] = aux.best_estimator_
        models[name]['mae'] = aux.best_score_
        models[name]['cols_order'] = os_X_tt.columns.values
        selection_time = time.time() - t_beg

        models[name]['selection_time'] = selection_time

        sample_f_path = f'{name}_{now_date.strftime("%Y%m%d-%H%M")}.sav'

        print(f"Saving model at {sample_f_path}")    
        joblib.dump(models[name]['bestModel'], sample_f_path)

        print(f"El tiempo de seleccion fue: {selection_time:0.3f} s")
        print(f"El error f_score de la familia {name} es: {models[name]['mae']:0.3f}")
        print('*'*80)

    mod_name = None
    best_mae = -np.inf
    for name in models:
        if models[name]['mae'] > best_mae:
            mod_name = name
            best_mae = models[name]['mae']

    print(f"best model: " + mod_name + " with an error of: " + str(best_mae))

    return models,mod_name

In [ ]:
# def all_years(fecha_ini, fecha_fin, intervalo, X, y):
def all_years(fecha_ini, fecha_fin, intervalo,cluester1_all_no,label = 'label'):
    #Completar los datos
    minimo = fecha_ini
    maximo= fecha_fin
    resultados_final = pd.DataFrame()
    #ix = pd.DatetimeIndex(start=datetime(diasconsulta.year, diasconsulta.month, diasconsulta.day, 0,0,0), end=datetime(diasconsulta.year, diasconsulta.month, diasconsulta.day,23,0,0), freq='H')
    ix = pd.date_range(start=dt.datetime(minimo.year, minimo.month, minimo.day,0,0,0), end=dt.datetime(maximo.year, maximo.month, maximo.day,0,0,0), freq=intervalo)
    for year in ix:
        print('*************')
        inicio = dt.datetime(year.year-5, 1, 1)
        fin = year
        
        fin2 = dt.datetime(year.year+2, 1, 1)
        print(inicio)
        print(fin)
        print(fin2)
        train = cluester1_all_no[(cluester1_all_no['date']>=inicio)&(cluester1_all_no['date']<=fin)]
        test = cluester1_all_no[(cluester1_all_no['date']>fin)&(cluester1_all_no['date']<fin2)]
        train.set_index(['pair','date'], inplace = True)
        test.set_index(['pair','date'], inplace = True)
        X_train = train.drop(columns = [label])
        Y_train = train[label]
        X_train_selected = select_features(X_train, Y_train)
        X_test = test[X_train_selected.columns].copy()
        Y_test = test[label]
        now_date = dt.date.today()
        n_proc = 3
        models_best,model_name = grid(now_date, n_proc, X_train_selected, Y_train, models, score = 'roc_auc', cv = 5)
        y_pred =  models_best[model_name]['bestModel'].predict_proba(X_test)
        X_test_tot = X_test.copy()
        X_test_tot['Pred'] = y_pred[:,1]
        X_test_tot['Real'] = test[label]
        resultados_final = pd.concat([resultados_final,X_test_tot[['Pred','Real']]])
    return resultados_final

In [ ]:
fecha_ini = dt.datetime(2014, 1, 1)
fecha_fin = dt.datetime(2019, 1, 1)
intervalo = 'Y'
total = all_years(fecha_ini, fecha_fin, intervalo,cluester1_all_no)

In [ ]:
total['Real'].sum()

In [ ]:
total.to_csv('cluesterAll_final_result.csv')

In [ ]:
for k, v in total.groupby('pair'):
    print(k)

### Portafolio

In [ ]:
cluster1=['MCHI US Equity','EWH US Equity','EWY US Equity'] #asia   
cluster2=['EWQ US Equity','EWG US Equity','EWU US Equity'] #europa   
cluster3=['XLI US Equity','XLB US Equity','XLF US Equity','XLE US Equity'] #USA sectores1
cluster4=['INDA US Equity','THD US Equity','EWM US Equity'] #asia superemergente
cluster5=['SPY US Equity','XLK US Equity','XLY US Equity'] #USA sectores2
clusterAll=['SPY US Equity','INDA US Equity','EWQ US Equity','XLF US Equity','MCHI US Equity'] #Uno de todos

In [ ]:
cluster=clusterAll

In [ ]:
dfprobv=total.drop(columns = 'Real')
dfprobv=dfprobv.unstack().T
dfprobv.reset_index(inplace = True)
dfprobv.drop(['level_0'],axis=1,inplace = True)
dfprobv.set_index('date',inplace = True)
dfprobv.head()

In [ ]:
from itertools import combinations

comb = combinations(list(range(len(cluster))), 2)
etfs=[]
for par in list(comb): 
    etfs.append(cluster[par[0]]+'_'+cluster[par[1]])

    print(etfs)

dfprobv = dfprobv[etfs].copy()

In [ ]:
votosv=pd.DataFrame(0, index=dfprobv.index.tolist(), columns=cluster)

In [ ]:
comb = combinations(list(range(len(cluster))), 2)

j=0
for par in list(comb):
    for i in range(len(votosv)):
       
        if dfprobv.iloc[i,j]>0.6:
            votosv.iloc[i].loc[cluster[par[0]]]+=2
            
        elif dfprobv.iloc[i,j]<0.4:
            votosv.iloc[i].loc[cluster[par[1]]]+=2
            
        else: 
            votosv.iloc[i].loc[cluster[par[0]]]+=1
            votosv.iloc[i].loc[cluster[par[1]]]+=1
    j+=1
    
votosv= (votosv.T / votosv.T.sum()).T

### precios etfs

In [ ]:
file_name = 'all_info.csv'

In [ ]:
allinfotypes = {"date":str,
                "PX_LAST":float,
                "PX_OPEN":float,
                "PX_HIGH":float,
                "PX_LOW":float,
                "PX_VOLUME":float,
                "TOT_RETURN_INDEX_NET_DVDS":float,
                "etf":str}
rawdata = pd.read_csv(file_name,sep =",",encoding = 'UTF-8', dtype = allinfotypes )

In [ ]:
rawdata["date"] = pd.to_datetime(rawdata["date"],format = '%Y-%m-%d')

In [ ]:
columns_relevant = ["date","TOT_RETURN_INDEX_NET_DVDS","etf"]
datafil = rawdata[columns_relevant].copy()

In [ ]:
datafilind = datafil.set_index(['date','etf']).unstack()
cols = [f"{l1}" for (l0, l1) in datafilind.columns]
datafilind.columns = cols
datafilind.reset_index(inplace = True)

In [ ]:
#votosv.data_to_prepros2[data_to_prepros2.index.dayofweek==2]
#votosv[votosv.index.dayofweek==2]

In [ ]:
data_to_prepros2 = datafilind[['date']+cluster].copy()
data_to_prepros2.set_index('date',inplace = True)
data_to_prepros2=data_to_prepros2[data_to_prepros2.index.dayofweek==2] #miercoles
votosv=votosv[votosv.index.dayofweek==2] #miercoles

for i in cluster:
    name_col = i +'_rt1d'
    data_to_prepros2[name_col] =data_to_prepros2[[i]]/data_to_prepros2[[i]].shift(periods=+1) - 1 #validar orden

data_to_prepros2=pd.merge(data_to_prepros2, votosv, left_index=True, right_index=True)
columns=data_to_prepros2.columns.tolist()
data_to_prepros2=data_to_prepros2[columns[:len(columns)-len(cluster)]]
data_to_prepros2['strategy']=pd.DataFrame(data_to_prepros2[data_to_prepros2.columns[len(cluster):len(cluster)*2]].values*votosv.shift(1).values).T.sum().tolist() #shift para multiplicar portafolio por retornos del proximo dia
data_to_prepros2['benchmark']=pd.DataFrame(data_to_prepros2[data_to_prepros2.columns[len(cluster):len(cluster)*2]].values*(1/len(cluster))).T.sum().tolist()
data_to_prepros2['strategy']=(data_to_prepros2[['strategy']]+1)
data_to_prepros2['benchmark']=(data_to_prepros2[['benchmark']]+1)
data_to_prepros2.iloc[0].loc['strategy']=data_to_prepros2.iloc[0].loc['benchmark']
#data_to_prepros2['strategy']=data_to_prepros2[['strategy']]*data_to_prepros2[['strategy']].shift(periods=+1)
#data_to_prepros2['benchmark']=data_to_prepros2[['benchmark']]*data_to_prepros2[['benchmark']].shift(periods=+1)

In [ ]:
for i in range(len(data_to_prepros2)):
    if i!=0:
        data_to_prepros2.iloc[i].loc['strategy']=data_to_prepros2.iloc[i].loc['strategy']*data_to_prepros2.iloc[i-1].loc['strategy']
        data_to_prepros2.iloc[i].loc['benchmark']=data_to_prepros2.iloc[i].loc['benchmark']*data_to_prepros2.iloc[i-1].loc['benchmark']
data_to_prepros2.head()   

In [ ]:
df1 = pd.DataFrame({'date':['2014-12-31'],'strategy': [1000],'benchmark': [1000]})
df1["date"] = pd.to_datetime(df1["date"],format = '%Y-%m-%d')
df1.set_index('date',inplace = True)
df2=pd.DataFrame([data_to_prepros2['strategy']*1000,data_to_prepros2['benchmark']*1000]).T
#df2.drop(df2.index[0],inplace = True)
result=pd.concat([df1, df2])

In [ ]:
plt.figure(figsize=(16, 6))
sns.set(style='darkgrid')
sns.lineplot(x=result.index.tolist(),y=result[result.columns[0]],label=result.columns[0])
sns.lineplot(x=result.index.tolist(),y=result[result.columns[1]],label=result.columns[1])
plt.ylabel("price");

In [ ]:
#votosv.to_csv('votosc1.csv')

In [ ]:
labelzona=['AAXJ US Equity']
data_to_prepros3 = pd.DataFrame(datafilind[['date']+labelzona])
data_to_prepros3.set_index('date',inplace = True)

data_to_prepros3['rt1d'] =data_to_prepros3[labelzona]/data_to_prepros3[labelzona].shift(periods=+1) #validar orden

data_to_prepros3=pd.merge(data_to_prepros3, votosv, left_index=True, right_index=True)
columns=data_to_prepros3.columns.tolist()
data_to_prepros3=data_to_prepros3.drop(columns=votosv.columns.tolist())
#data_to_prepros2['strategy']=data_to_prepros2[['strategy']]*data_to_prepros2[['strategy']].shift(periods=+1)
#data_to_prepros2['benchmark']=data_to_prepros2[['benchmark']]*data_to_prepros2[['benchmark']].shift(periods=+1)

In [ ]:
for i in range(len(data_to_prepros3)):
    if i!=0:
        data_to_prepros3.iloc[i].loc['rt1d']=data_to_prepros3.iloc[i].loc['rt1d']*data_to_prepros3.iloc[i-1].loc['rt1d']
data_to_prepros3.head()   

In [ ]:
df3 = pd.DataFrame({'date':['2014-12-31'],'rt1d': [1000]})
df3["date"] = pd.to_datetime(df3["date"],format = '%Y-%m-%d')
df3.set_index('date',inplace = True)
df4=pd.DataFrame([data_to_prepros3['rt1d']*1000]).T
#df2.drop(df2.index[0],inplace = True)
result2=pd.concat([df3, df4])

In [ ]:
plt.figure(figsize=(16, 6))
sns.set(style='darkgrid')
sns.lineplot(x=result.index.tolist(),y=result[result.columns[0]],label=result.columns[0])
sns.lineplot(x=result.index.tolist(),y=result[result.columns[1]],label=result.columns[1])
sns.lineplot(x=result.index.tolist(),y=result2[result2.columns[0]],label=labelzona[0])
plt.ylabel("price");